In [1]:
import pandas as pd
import numpy as np


In [2]:
data=pd.read_csv('Tweets.csv')
data.head(1)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)


In [3]:
data1=data[['airline_sentiment','text']]

In [4]:
data1.columns=['SENTIMENT','TWEET']
df=data1.copy()
df=df.loc[(df['SENTIMENT']=='negative') | (df['SENTIMENT']=='positive')]

In [5]:
df['SENTIMENT'].value_counts()

negative    9178
positive    2363
Name: SENTIMENT, dtype: int64

In [6]:
df.isnull().sum()

SENTIMENT    0
TWEET        0
dtype: int64

In [7]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string


In [8]:
wnl=WordNetLemmatizer()

In [9]:
pd.set_option('display.max_colwidth',None)
pd.set_option('display.max_columns',None)

In [10]:
df['TWEET'].head(5)

1                                                                    @VirginAmerica plus you've added commercials to the experience... tacky.
3              @VirginAmerica it's really aggressive to blast obnoxious "entertainment" in your guests' faces &amp; they have little recourse
4                                                                                     @VirginAmerica and it's a really big bad thing about it
5    @VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's really the only bad thing about flying VA
6                                                             @VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)
Name: TWEET, dtype: object

In [11]:
x=df['TWEET'].iloc[0].split()
print(x)
s=set(stopwords.words('english'))
l1=[]
for i in x:
    if i not in s:
        l1.append(wnl.lemmatize(i.lower()))
s1=' '.join(l1)
print(s1)
        

['@VirginAmerica', 'plus', "you've", 'added', 'commercials', 'to', 'the', 'experience...', 'tacky.']
@virginamerica plus added commercial experience... tacky.


In [12]:
def remove_mentionsandlinks(text):
    text=re.sub(r'@[A-Za-z0-9]+','',text.lower())   #removing mentions
    text=re.sub(r'#','',text)   #removing '#' symbol
    text=re.sub(r'https?:\/\/\S+','',text)  # removing links
    text=re.sub(r'[^A-Za-z0-9]',' ',text)
    return text

In [13]:
df['TWEET']=df['TWEET'].apply(remove_mentionsandlinks)


In [14]:
df.head(1)

,SENTIMENT,TWEET
1,positive,plus you ve added commercials to the experience tacky


In [15]:
s=set(stopwords.words('english'))
def stopwords_and_stemming(text):
    l2=[]
    for i in text.split():
        
        if i not in s:
            l2.append(wnl.lemmatize(i))
    s2=' '.join(l2)
    return s2
#removing stopwords and lemmatazing

In [16]:
df['TWEET']=df['TWEET'].apply(stopwords_and_stemming)

In [18]:
df.head(1)

,SENTIMENT,TWEET
1,positive,plus added commercial experience tacky


In [19]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(df['SENTIMENT'])

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
sparse=tfidf.fit_transform(df['TWEET'])

In [21]:
vector1=sparse.toarray()

In [22]:
from sklearn.model_selection import train_test_split
x1_train,x1_test,y1_train,y1_test=train_test_split(vector1,y,test_size=0.15,random_state=0)

In [23]:
from sklearn.svm import SVC
from sklearn.metrics import  roc_auc_score
c1=SVC(kernel='linear',random_state=0)
c1.fit(x1_train,y1_train)
y1_train_pred=c1.predict(x1_train)
y1_test_pred=c1.predict(x1_test)
print('the roc_auc_score for training set is-------> ', roc_auc_score(y1_train,y1_train_pred))
print('the roc_auc_score for testing set is-------> ', roc_auc_score(y1_test,y1_test_pred))

the roc_auc_score for training set is------->  0.9256989191751207
the roc_auc_score for testing set is------->  0.8258365568513538


In [24]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(x1_train,y1_train)
print('the roc_auc_score for training set is-------> ', roc_auc_score(y1_train,lr.predict(x1_train)))
print('the roc_auc_score for testing set is-------> ', roc_auc_score(y1_test,lr.predict(x1_test)))

the roc_auc_score for training set is------->  0.8166718975962602
the roc_auc_score for testing set is------->  0.7725442544685868


In [25]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()
mnb.fit(x1_train,y1_train)
print('the roc_auc_score for training set is-------> ', roc_auc_score(y1_train,mnb.predict(x1_train)))
print('the roc_auc_score for testing set is-------> ', roc_auc_score(y1_test,mnb.predict(x1_test)))

the roc_auc_score for training set is------->  0.6906677983336301
the roc_auc_score for testing set is------->  0.6369972903960048


In [26]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
gnb.fit(x1_train,y1_train)
print('the roc_auc_score for training set is-------> ', roc_auc_score(y1_train,gnb.predict(x1_train)))
print('the roc_auc_score for testing set is-------> ', roc_auc_score(y1_test,gnb.predict(x1_test)))

the roc_auc_score for training set is------->  0.9006923964610848
the roc_auc_score for testing set is------->  0.6759650524767506


## SUPPORT VECTOR CLASSIFIER
#### the roc_auc_score for training set is------->  0.9256989191751207
#### the roc_auc_score for testing set is------->  0.8258365568513538

## LogisticRegression
#### the roc_auc_score for training set is------->  0.8166718975962602
#### the roc_auc_score for testing set is------->  0.7725442544685868

## MultinomialNB
#### the roc_auc_score for training set is------->  0.6906677983336301
#### the roc_auc_score for testing set is------->  0.6369972903960048

## GaussianNB
#### the roc_auc_score for training set is------->  0.9006923964610848
#### the roc_auc_score for testing set is------->  0.6759650524767506